In [1]:
import sys

from os import getcwd
from os.path import dirname, abspath


sys.path.append(dirname((abspath(getcwd()))))

In [2]:
from warnings import warn

import numpy as np
import pandas as pd

# Import pipeline
from sklearn.pipeline import Pipeline
# Import classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
# Import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# Import the hashing vectorizer
from sklearn.feature_extraction.text import HashingVectorizer
# Import other preprocessing modules
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import chi2, SelectKBest
# Import functional utilities
from sklearn.preprocessing import FunctionTransformer, MaxAbsScaler
from sklearn.pipeline import FeatureUnion

from school_budget.data import multilabel_sample_dataframe, multilabel_train_test_split, get_data_by_columns
from school_budget.config import NUMERIC_COLUMNS, LABELS
from school_budget.features import combine_text_columns, SparseInteractions

In [3]:
df = pd.read_csv("../datasets/TrainingData.csv", index_col=0)
NON_LABELS = [c for c in df.columns if c not in LABELS]
SAMPLE_SIZE = 40000
sampling = multilabel_sample_dataframe(
    df,
    pd.get_dummies(df[LABELS]),
    size=SAMPLE_SIZE,
    min_count=25,
    seed=43
)
dummy_labels = pd.get_dummies(sampling[LABELS])
X_train, X_test, y_train, y_test = multilabel_train_test_split(
    sampling[NON_LABELS],
    dummy_labels,
    0.2,
    min_count=3,
    seed=43
)

In [4]:
# Select 300 best features
chi_k = 300
# Perform preprocessing
get_text_data = FunctionTransformer(combine_text_columns, validate=False)
get_numeric_data = FunctionTransformer(get_data_by_columns, validate=False)

# Create the token pattern: TOKENS_ALPHANUMERIC
TOKENS_ALPHANUMERIC = '[A-Za-z0-9]+(?=\\s+)'

# Instantiate pipeline: pl
pl = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', SimpleImputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', CountVectorizer(token_pattern=TOKENS_ALPHANUMERIC,
                                                   ngram_range=(1, 2))),
                    ('dim_red', SelectKBest(chi2, k=chi_k))
                ]))
             ]
        )),
        ('scale', MaxAbsScaler()),
        ('clf', OneVsRestClassifier(LogisticRegression()))
    ])

In [ ]:
# Instantiate the winning model pipeline: pl
pl = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', SimpleImputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', HashingVectorizer(token_pattern=TOKENS_ALPHANUMERIC,
                                                     norm=None, binary=False, alternate_sign=False,
                                                     ngram_range=(1, 2))),
                    ('dim_red', SelectKBest(chi2, k=chi_k))
                ]))
             ]
        )),
        ('int', SparseInteractions(degree=2)),
        ('scale', MaxAbsScaler()),
        ('clf', OneVsRestClassifier(LogisticRegression(max_iter=100)))
    ])
pl.fit(X_train, y_train)

/Users/slaily/.local/share/virtualenvs/school_budget-dhmayhNO/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/slaily/.local/share/virtualenvs/school_budget-dhmayhNO/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver opti

/Users/slaily/.local/share/virtualenvs/school_budget-dhmayhNO/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/slaily/.local/share/virtualenvs/school_budget-dhmayhNO/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver opti

/Users/slaily/.local/share/virtualenvs/school_budget-dhmayhNO/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/slaily/.local/share/virtualenvs/school_budget-dhmayhNO/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver opti

In [ ]:
predictions = pl.predict_proba(X_test)
# Format correctly in new DataFrame: prediction_df
prediction_df = pd.DataFrame(columns=pd.get_dummies(df[LABELS]).columns,
                             index=X_test.index,
                             data=predictions)


# Save prediction_df to csv called "predictions.csv"
prediction_df.to_csv("predictions.csv")

In [ ]:
top_n_pred = np.argsort(predictions, axis=1)[:,-3:]
class_labels = pl.classes_
print(class_labels)
print(class_labels[top_n_pred])
print(prediction_df.columns[52])
print(prediction_df.columns[98])
predicted_sorted = predictions[np.arange(top_n_pred.shape[0]),top_n_pred.T].T
print(predicted_sorted[:,-3:])

In [ ]:
from joblib import dump
dump(pl, "model-1.1.0.joblib")

In [ ]:
f_row = X_test.iloc[0].to_frame()
print(X_test.iloc[0].to_frame())
predictions = pl.predict_proba(X_test.head(1))
print(type(X_test.head(1)))
print(X_test.head(1).columns)
print(X_test.iloc[0].to_frame().T.columns)
print(predictions)
print(predictions.shape)
# pl.predict_proba(f_row)